# A/B Testing

## Test of Means Analysis
Statistical calculation of whether the mean values of the treatment and control groups are the same

## T-Test
Give p-value - likelihood the actual difference between the means is zero.  
p-value < 0.05 (statistically significant)  
T-test is important when working in small data set

There are three types of t-tests you can use:  
1. Paired  
2. Equal variance  
3. Unequal variance  

In a random experiment you will usually assume that variances between the groups are different, so we’ll use an unequal variance t-test.

In [ ]:
# load package
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.neighbors import KDTree, NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

## Supper Hero Data

In [ ]:
super_hero = pd.read_excel('superherodata.xlsx')
super_hero

In [ ]:
useful_columns = list(super_hero.columns)[:8]
useful_columns

In [ ]:
super_hero_clean_data = super_hero[useful_columns].dropna()
super_hero_clean_data

In [ ]:
good = super_hero_clean_data[super_hero_clean_data['Alignment']=='good'].select_dtypes(exclude='O')
good

In [ ]:
bad = super_hero_clean_data[super_hero_clean_data['Alignment']=='bad'].select_dtypes(exclude='O')
bad

In [ ]:
sp = ttest_ind(good, bad, equal_var=False)
print('Statistic:', sp.statistic)
print('pvalue:', sp.pvalue)

## Customer Support Time Study Data

In [ ]:
customer_data = pd.read_excel('customersupporttimestudydata.xlsx', header=2)
customer_data

In [ ]:
customer_data = customer_data.dropna().drop(index=8).drop(columns='Customer Onboarding Process').astype('float')
customer_data

In [ ]:
nataly = customer_data[:4].sum().values
joe = customer_data[4:].sum().values
joe

In [ ]:
sp = ttest_ind(joe, nataly, equal_var=False)
print('Statistic:', sp.statistic)
print('pvalue:', sp.pvalue)

## Customer Service AB Testing Data 

In [ ]:
customer_service_data = pd.read_csv('customerserviceabtestdata.csv')
customer_service_data.head()

In [ ]:
phone_count = customer_service_data['Phone Number'].value_counts()
phone_count[:10]

In [ ]:
AutomatedFlag_0 = customer_service_data[customer_service_data['AutomatedFlag']==0]['CS Rating'].values
AutomatedFlag_1 = customer_service_data[customer_service_data['AutomatedFlag']==1]['CS Rating'].values
AutomatedFlag_1

In [ ]:
sp = ttest_ind(AutomatedFlag_0, AutomatedFlag_1, equal_var=False)
print('Statistic:', sp.statistic)
print('pvalue:', sp.pvalue)

## Grocery Website AB Test Data

In [ ]:
data = pd.read_csv('grocerywebsiteabtestdata.csv')
data.head()

In [ ]:
data = data.drop(columns='RecordID')

In [ ]:
data['Group'] = np.where(data['ServerID'] == 1, 'Treatment', 'Control')
data.head()

In [ ]:
data_summary = data.groupby(['IP Address', 'LoggedInFlag', 'Group', 'ServerID'], as_index=False).sum('VisitPageFlag')
data_summary.head(10)

In [ ]:
# Multiple visit fix
data_summary['VisitPageFlag'] = np.where(data_summary['VisitPageFlag'] == 0, 0, 1)
data_summary.head()

In [ ]:
non_logged_user = data_summary[data_summary['LoggedInFlag'] != 1]
non_logged_user.head(10)

In [ ]:
treatment = non_logged_user[non_logged_user['Group']=='Treatment']['VisitPageFlag'].values
control = non_logged_user[non_logged_user['Group']=='Control']['VisitPageFlag'].values
control

In [ ]:
sp = ttest_ind(control, treatment, equal_var=False)
print('Statistic:', sp.statistic)
print('pvalue:', sp.pvalue)

In [ ]:
non_logged_user.head()

In [ ]:
summary_non_logged_user = non_logged_user.groupby(['Group', 'VisitPageFlag'], as_index=False).size()
summary_non_logged_user

In [ ]:
perc_control_increase = 6131 / (6131 + 26839) * 100
perc_treatment_increase = 3847 / (3847 + 12696) * 100

print('Control Increament:', perc_control_increase)
print('Treatment Increament:', perc_treatment_increase)

In [ ]:
print('Conclusion:', perc_treatment_increase - perc_control_increase, '% jump if company change link to picture image of the app')

## Matched Pair Design

### New Product Control Data

In [ ]:
new_product_control_data = pd.read_csv('newproductcontroldata.csv')
new_product_control_data.head()

In [ ]:
cherry_product_treatment_stores = pd.read_csv('cherry-product-treatment-stores.csv')
cherry_product_treatment_stores

In [ ]:
useful_fields_names = ['Category Sales', 'Product Count']
state_name = 'CA'
state_CA = new_product_control_data.query('State == @state_name').reset_index(drop=True)
state_CA.head()

In [ ]:
tree = KDTree(state_CA[useful_fields_names], leaf_size=2)

for store_id in cherry_product_treatment_stores['Store ID'].values:
    store_data = cherry_product_treatment_stores.query('`Store ID` == @store_id')[useful_fields_names]
    dist, ind = tree.query(store_data, k=3)
    stores_matched = state_CA.iloc[ind[0]][["Store ID", "State", "Category Sales", "Product Count"]]

    print(f'Distance - {dist[0]}')
    print(f'Matched Stores:\n {stores_matched} \n')

In [ ]:
nn = NearestNeighbors()
nn.fit(state_CA[useful_fields_names])

for store_id in cherry_product_treatment_stores['Store ID'].values:
    store_data = cherry_product_treatment_stores.query('`Store ID` == @store_id')[useful_fields_names]
    dist, ind = nn.kneighbors(store_data, 3)
    stores_matched = state_CA.iloc[ind[0]][["Store ID", "State", "Category Sales", "Product Count"]]

    print(f'Distance - {dist[0]}')
    print(f'Matched Stores:\n {stores_matched} \n')

In [ ]:
from scipy.spatial import KDTree as scipy_KDTree

In [ ]:
scipy_kdtree = scipy_KDTree(state_CA[useful_fields_names])

for store_id in cherry_product_treatment_stores['Store ID'].values:
    store_data = cherry_product_treatment_stores.query('`Store ID` == @store_id')[useful_fields_names]
    dist, ind = scipy_kdtree.query(store_data, k=3)
    stores_matched = state_CA.iloc[ind[0]][["Store ID", "State", "Category Sales", "Product Count"]]

    print(f'Distance - {dist[0]}')
    print(f'Matched Stores:\n {stores_matched} \n')

### Scaling the numerical features lead to better KD Tree results

In [ ]:
# Scale the features
transformer = ColumnTransformer(
    [
        ('scaler' , StandardScaler(), useful_fields_names),
    ], 
    remainder='drop'
)

transformer.fit(state_CA)

scaled_data = transformer.transform(state_CA)
scaled_data

In [ ]:
tree_scaler = KDTree(scaled_data, leaf_size=2)

for store_id in cherry_product_treatment_stores['Store ID'].values:
    store_data = cherry_product_treatment_stores.query('`Store ID` == @store_id')
    dist, ind = tree_scaler.query(transformer.transform(store_data), k=3)
    stores_matched = state_CA.iloc[ind[0]][["Store ID", "State", "Category Sales", "Product Count"]]

    print(f'Distance - {dist[0]}')
    print(f'Matched Stores:\n {stores_matched} \n')